
その四　声優の声から特徴を抽出する
声優の声データからMFCC（声の特徴量）を抽出し、表にまとめる。

In [9]:
import os
import numpy as np
import pandas as pd
import librosa
from IPython.display import clear_output

X_data = []  # 特徴行列
y_data = []  # クラスラベルデータ
speakers_number = 7 # 声優の人数
files_number = 100 # 一人あたりのサンプルの数

for speaker_num in range(1, speakers_number + 1):  # 声優フォルダの数だけ繰り返し処理
    clear_output()
    # ボイスサンプルがあるフォルダ名
    dir_name = f'VoiceSample/{str(speaker_num).zfill(3)}'
    file_list = os.listdir(dir_name)
    for i in range(files_number):
        clear_output()
        print(f"{int((i+(speaker_num-1)*files_number)/(files_number*speakers_number)*100)}%完了")
        print(str(speaker_num) + "人目のデータを処理中・・・")
        print(f"（進行度:{i+1}/{files_number}ファイル）" )
        file_path = os.path.join(dir_name, file_list[i])  # 音声ファイルへのパス
        y, sr = librosa.load(file_path)  # 音声ファイルを読み込む
        mfcc = librosa.feature.mfcc(y, sr)  # MFCC
        mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
        mfcc = mfcc.flatten()
        mfcc = mfcc.tolist()
        mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
        X_data.append(mfcc)
        y_data.append(speaker_num)

X = pd.DataFrame(X_data, columns=[f'mfcc_{n}' for n in range(1, 13)]) # mfcc_1,mfcc_2,..のように行のタイトルをつける
y = pd.DataFrame({'target': y_data}) # targetタイトルを追加

df = pd.concat([X, y], axis=1) # 合体！！！！！！
df.to_csv('mfcc.csv', index=False)  # csvで保存
df.head() # 見せてほしいな

99%完了
7人目のデータを処理中・・・
（進行度:100/100ファイル）


/tmp/ipykernel_8879/2882801715.py:26: FutureWarning: Pass y=[-3.0901801e-06 -4.7229732e-06  1.2742006e-05 ... -3.3230467e-06
 -1.4371163e-06  0.0000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y, sr)  # MFCC


,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,target
0,79.305939,-2.388980,28.821922,8.106466,-7.335474,-5.098837,2.708056,1.108589,-6.039336,-11.022418,8.562423,-8.208080,1
1,46.389778,15.316972,29.470976,15.136189,7.453907,-4.679848,6.583255,-3.255087,2.030280,-12.806031,9.289038,-7.932013,1
2,97.034988,19.946007,42.861015,15.739766,-5.527338,0.597292,3.899351,-1.708565,-6.449223,-6.763590,11.951451,-12.204566,1
3,84.402969,14.758104,36.634586,19.574112,-5.123018,-3.361386,4.112405,-0.293475,-1.699620,-10.678855,12.195671,-13.852660,1
4,94.733459,15.617436,38.703354,16.677921,-3.071551,-1.830640,5.379351,0.139020,-8.512769,-13.017569,8.190866,-15.577478,1


機械学習で解決する対象の問題には分類問題と回帰問題の二種類がある。
分類問題はあるデータからそのデータがどのタイプに属するかを予測する。例えば、レストランの客のデータからその人がそのレストランを好きか嫌いかを予測する場合は分類問題である。
回帰問題はあるデータからある値を予測する。例えば、レストランの客のデータからその人がそのレストランにあと何回訪れるかを予測する場合は回帰問題である。
pycaretでは分類問題を扱う場合はclassification, 回帰問題を扱う場合はregressionを使用する。（この違いがわかっていなかったため我々は最初regressionでしてしまっていた）


その五　声優の声を機械学習させる

1.データの前準備

Pycaretの関数を使う場合以下のセルを実行してから使うこと

In [1]:
import pandas as pd
from pycaret.classification import *

print("機械学習の準備中・・・")

mfcc_data = pd.read_csv("mfcc.csv") # MFCCのデータを読み込む
# データの前準備
reg = setup(data=mfcc_data, target='target', data_split_shuffle=True, silent=True, use_gpu=True, fold=3, n_jobs=-1)

print("準備完了")

,Description,Value
0,session_id,7796
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(700, 13)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


準備完了


2.モデルの作成

In [32]:
# 機械学習のモデルを複数比較し、精度が高かったモデルを表示する。
print("最適モデルを検索中・・・")

compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.6462,0.9170,0.6486,0.6479,0.6445,0.5873,0.5881,4.0500
qda,Quadratic Discriminant Analysis,0.6442,0.9086,0.6453,0.6701,0.6494,0.5847,0.5870,1.4150
rf,Random Forest Classifier,0.6381,0.9100,0.6402,0.6389,0.6364,0.5778,0.5784,6.8900
lr,Logistic Regression,0.6197,0.8970,0.6223,0.6220,0.6189,0.5563,0.5568,4.2900
lightgbm,Light Gradient Boosting Machine,0.6095,0.8954,0.6115,0.6194,0.6113,0.5443,0.5451,3.9900
lda,Linear Discriminant Analysis,0.5951,0.8939,0.5985,0.5988,0.5935,0.5277,0.5286,0.1450
gbc,Gradient Boosting Classifier,0.5911,0.8810,0.5937,0.5978,0.5915,0.5229,0.5237,5.8850
nb,Naive Bayes,0.5828,0.8783,0.5848,0.5892,0.5822,0.5133,0.5143,0.2900
ridge,Ridge Classifier,0.5502,0.0000,0.5534,0.5550,0.5407,0.4754,0.4785,0.1250
knn,K Neighbors Classifier,0.5276,0.8455,0.5314,0.5234,0.5196,0.4491,0.4510,8.6400


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=8159, verbose=0,
                     warm_start=False)

3.モデルの微調整
4.予測

In [12]:
from pycaret.classification import *
# 最も精度が高かったモデルを生成
tuned_model = False #モデルをチューニングするか
final_model = True #モデルをファイナライズするか

print("モデルを作成中・・・")
model = create_model('et', verbose=False) # モデルを生成
print("モデルを調整中・・・")
if (tuned_model) :
    tuned_model = tune_model(model, optimize="Accuracy") # モデルを調整...すると精度落ちるの何...しないほうがいいのか？
print("テストデータで予測中・・・")
predict_data = predict_model(tuned_model or model, raw_score=True) # 既存のデータで予測 raw_scoreをTrueにしないとIndexErrorが出る（おそらく0ではなく1から始めているから）
predict_data.to_csv('predict_result.csv', index=False)  # csvで保存
if (final_model) :
    print("モデルを保存中・・・")
    final_model = finalize_model(model)
    save_model(final_model,'Final Model')#モデルを保存


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5644,0.8873,0.5541,0.5757,0.5573,0.4892,0.4940
1,0.6687,0.9304,0.6605,0.6696,0.6492,0.6118,0.6176
2,0.7239,0.9116,0.7239,0.7281,0.7157,0.6769,0.6797
Mean,0.6524,0.9098,0.6461,0.6578,0.6407,0.5926,0.5971
Std,0.0661,0.0176,0.0701,0.0628,0.0649,0.0778,0.0772


テストデータで予測中・・・


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.6066,0.9123,0.6286,0.6674,0.6070,0.5444,0.5546


モデルを保存中・・・
Transformation Pipeline and Model Successfully Saved


In [1]:
from pycaret.classification import *

final_model = load_model('Final Model') # モデルの読み込み

Transformation Pipeline and Model Successfully Loaded


In [5]:
import sounddevice as sd
import wave
import numpy as np

#音声を録音

FILE_NAME = 'my_recording.wav'  # 保存するファイル名
wave_length = 5  # 録音する長さ（秒）
sample_rate = 16_000  # サンプリング周波数

print("録音開始")
# 録音開始（wave_length秒間録音。wait で録音し終わるまで待つ）
data = sd.rec(int(wave_length * sample_rate), sample_rate, channels=1)
sd.wait()
print("録音終了")

# ノーマライズ。量子化ビット16bitで録音するので int16 の範囲で最大化する
data = data / data.max() * np.iinfo(np.int16).max

# float -> int
data = data.astype(np.int16)

# ファイル保存
with wave.open(FILE_NAME, mode='wb') as wb:
    wb.setnchannels(1)  # モノラル
    wb.setsampwidth(2)  # 16bit=2byte
    wb.setframerate(sample_rate)
    wb.writeframes(data.tobytes())  # バイト列に変換

録音開始
録音終了


In [6]:
from pycaret.classification import *
import pandas as pd
import librosa
import numpy as np

###MFCCを抽出###
print("声の特徴を抽出しています・・・")

data = []
TEST_FILE_NAME = "TestSample/hanae2.wav"

test = False

y, sr = librosa.load(TEST_FILE_NAME if test else FILE_NAME)  # 音声ファイルを読み込む
mfcc = librosa.feature.mfcc(y, sr)  # MFCC
mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
mfcc = mfcc.flatten()
mfcc = mfcc.tolist()
mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
data.append(mfcc)

df = pd.DataFrame(data, columns=[f'mfcc_{n}' for n in range(1, 13)])

df.to_csv('mfcc_my_recording.csv', index=False)  # csvで保存
df.head()

#録音した音声データからどの声優に似ているかを特定

actors = ["下野紘", "花江夏樹", "梶裕貴", "沢城みゆき", "鬼頭明里", "水瀬いのり", "悠木碧"]

predict_my_recording = predict_model(final_model, data = pd.read_csv("mfcc_my_recording.csv"), raw_score=True) # 未知データを予測
predict_label = predict_my_recording.at[0, "Label"] # 予測データを取得（小数点数）
print("あなたの声は" + actors[predict_label-1] + "に似ています。")

声の特徴を抽出しています・・・
あなたの声は水瀬いのりに似ています。


In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from IPython.display import clear_output

#声優の声データからMFCC（声の特徴量）を抽出し、表にまとめる。

X_data = []  # 特徴行列
y_data = [] #正解データ
speakers_number = 7 # 声優の人数
files_number = 5 #声優一人あたりのファイル数

for speaker_num in range(1, speakers_number + 1):  # 声優フォルダの数だけ繰り返し処理
    # ボイスサンプルがあるフォルダ名
    dir_name = f'TestSample/{str(speaker_num).zfill(3)}'
    file_list = os.listdir(dir_name)
    for i in range(files_number):
        clear_output()
        print(f"{int((i+(speaker_num-1)*files_number)/(files_number*speakers_number)*100)}%完了")
        print(str(speaker_num) + "人目のデータを処理中・・・")
        print(f"（進行度:{i+1}/{files_number}ファイル）" )
        file_path = os.path.join(dir_name, file_list[i])  # 音声ファイルへのパス
        y, sr = librosa.load(file_path)  # 音声ファイルを読み込む
        mfcc = librosa.feature.mfcc(y, sr)  # MFCC
        mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
        mfcc = mfcc.flatten()
        mfcc = mfcc.tolist()
        mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
        X_data.append(mfcc)
        y_data.append(speaker_num)

X = pd.DataFrame(X_data, columns=[f'mfcc_{n}' for n in range(1, 13)]) # mfcc_1,mfcc_2,..のように行のタイトルをつける
y = pd.DataFrame({'target': y_data}) # targetタイトルを追加

df = pd.concat([X, y], axis=1) # 合体！！！！！！
df.to_csv('mfcc_test.csv', index=False)  # csvで保存
df.head() # 見せてほしいな

In [13]:
from pycaret.classification import *
import numpy as np
import pandas as pd

actors = ["下野紘", "花江夏樹", "梶裕貴", "沢城みゆき", "鬼頭明里", "水瀬いのり", "悠木碧"]
speakers_number = 7

final_model = load_model('Final Model')
results = []
count = 0 #正解数
total = 0 #総数

predict_data = predict_model(final_model, data = pd.read_csv("mfcc_test.csv"), raw_score=True) # 未知データを予測

for predict_result, target in zip(predict_data["Label"], predict_data["target"]):
    result = [actors[int(predict_result-1)], predict_result, f'{actors[target-1]}({target})']
    if predict_result == target:
        count = count + 1
    total = total + 1
    results.append(result)

accuracy = count / total
print(f"正答率{int(accuracy*100)}%")
df = pd.DataFrame(results, columns=["予想した声優", "予測値", "正解値"])
df.to_csv("test_result.csv", index=False)
df.head()

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.4000,0.8348,0.4000,0.5509,0.4138,0.3000,0.3151


正答率40%


,予想した声優,予測値,正解値
0,花江夏樹,2,下野紘(1)
1,下野紘,1,下野紘(1)
2,悠木碧,7,下野紘(1)
3,花江夏樹,2,下野紘(1)
4,花江夏樹,2,下野紘(1)
